# Street canyon example in Radiosity

This example shows the process of simulating a geometry such as a street canyon while applying different properties to the wall boundaries. The setup of the canyon is comparable to **Sound propagation in street canyons: Comparison between
diffusely and geometrically reflecting boundaries** by Kang published in 2000.

Paper: https://doi.org/10.1121/1.428580


First, import all of our dependencies.

In [1]:
import numpy as np
import pyfar as pf
import sofar as sf
import sparrowpy as sp
import matplotlib.pyplot as plt
import os

%matplotlib inline
# %matplotlib ipympl

AttributeError: module 'numba' has no attribute 'prange'

Define the street canyon and set general parameters for the simulation.

In [ ]:
# canyon boundary dimensions in meters 120 20 18
length = 35  # X
width = 10  # Y
height = 4  # Z  (different heights were used by Kang)

# simulation parameters
patch_size = 4
etc_duration = 1
etc_time_resolution = 1 / 1000
max_reflection_order = 50
speed_of_sound = 343.2
absorption = 0.1

# create geometry for the two facades and the ground
boundaries = [
    sp.geometry.Polygon(
        np.array(
            [  # corner points
                [0, 0, 0],
                [0, 0, height],
                [length, 0, 0],
                [length, 0, height],
            ],
        ),
        np.array([1, 0, 0]),  # up vector
        np.array([0, 1, 0]),  # normal vector
    ),
    sp.geometry.Polygon(
        np.array(
            [
                [0, 0, 0],
                [0, width, 0],
                [length, 0, 0],
                [length, width, 0],
            ],
        ),
        np.array([1, 0, 0]),
        np.array([0, 0, 1]),
    ),
    sp.geometry.Polygon(
        np.array(
            [
                [0, width, 0],
                [length, width, 0],
                [0, width, height],
                [length, width, height],
            ],
        ),
        np.array([1, 0, 0]),
        np.array([0, -1, 0]),
    ),
]

# for example, set the same position for source and receiver
source = pf.Coordinates(30, 6, 1)
receiver = pf.Coordinates(30, 6, 1)

For further steps create the instance of the simulation class called ``DirectionalRadiosityFast`` from the previously defined boundaries and the patch size.
Two instances are created so that other properties can be compared later.

In [ ]:
radiosity_diffuse = sp.DirectionalRadiosityFast.from_polygon(
    boundaries,
    patch_size,
)
radiosity_retroreflection = sp.DirectionalRadiosityFast.from_polygon(
    boundaries,
    patch_size,
)

In the diffuse case, the scattering coefficient is set to 1 for each boundary.

For the retroreflection, BRDF data in the form of a ``.sofa`` file is used from the following paper: https://doi.org/10.3397/IN_2025_1074741

In [ ]:
%%capture
# create directional scattering data (totally diffuse) for one frequency band
brdf_sources = pf.Coordinates(0, 0, 1, weights=1)
brdf_receivers = pf.Coordinates(0, 0, 1, weights=1)
frequencies = np.array([4000])
brdf_diffuse = sp.brdf.create_from_scattering(
    brdf_sources,
    brdf_receivers,
    pf.FrequencyData(1, frequencies),  # diffuse scattering => 1
    pf.FrequencyData(absorption, frequencies),
)

# set the created scattering data in the radiosity instance
radiosity_diffuse.set_wall_brdf(
    np.arange(len(boundaries)).tolist(),
    brdf_diffuse,
    brdf_sources,
    brdf_receivers,
)


# use retroreflective scattering for the second simulation
BRDF_file = os.path.join(os.curdir, "brdf_walls_RISC.sofa")
sofa_data = sf.read_sofa(BRDF_file)
brdf_rr, brdf_sources_rr, brdf_receivers_rr = pf.io.convert_sofa(sofa_data)
brdf_sources_rr.weights = sofa_data.SourceWeights
brdf_receivers_rr.weights = sofa_data.ReceiverWeights
radiosity_retroreflection.set_wall_brdf(
    np.arange(len(boundaries)).tolist(),
    brdf_rr,
    brdf_sources,
    brdf_receivers,
)

The air attenuation can be defined as well. Here, it is set to 0.

In [ ]:
radiosity_diffuse.set_air_attenuation(
    pf.FrequencyData(
        np.zeros_like(brdf_diffuse.frequencies),
        brdf_diffuse.frequencies,
    ),
)

radiosity_retroreflection.set_air_attenuation(
    pf.FrequencyData(np.zeros_like(brdf_rr.frequencies), brdf_rr.frequencies),
)

To run the simulation, the geometry must first be calculated (baked) to enable the computation of energy exchange between patches at a later stage.

The patch to patch relationships contain the form factors, the geometrical relationship between the patches as well as the BRDFs.

This is the most heavy part of the simulation and is independent of the source and receiver positions.

In [ ]:
radiosity_diffuse.bake_geometry()
radiosity_retroreflection.bake_geometry()

Next, the distribution of the source's initial energy to each visible patch is simulated.

In [ ]:
radiosity_diffuse.init_source_energy(source)
radiosity_retroreflection.init_source_energy(source)

Subsequently, the energy exchange between the patches and the energy at the receiver is calculated.

In [ ]:
radiosity_diffuse.calculate_energy_exchange(
    speed_of_sound=speed_of_sound,
    etc_time_resolution=etc_time_resolution,
    etc_duration=etc_duration,
    max_reflection_order=max_reflection_order,
)
radiosity_retroreflection.calculate_energy_exchange(
    speed_of_sound=speed_of_sound,
    etc_time_resolution=etc_time_resolution,
    etc_duration=etc_duration,
    max_reflection_order=max_reflection_order,
)

Finally, the energy is collected at the specified receiver point(s).

In [ ]:
etc_diffuse = radiosity_diffuse.collect_energy_receiver_mono(
    receivers=receiver,
)
etc_retroreflection = radiosity_retroreflection.collect_energy_receiver_mono(
    receivers=receiver,
)

The two simulations can be visually compared.

In [ ]:
plt.figure()
pf.plot.time(
    etc_diffuse,
    dB=True,
    log_prefix=10,
    label="diffuse",
    linestyle="--",
)
pf.plot.time(
    etc_retroreflection[0, -1],  # plot only the highest frequency band (-1)
    dB=True,
    log_prefix=10,
    label="retroreflection",
)

plt.legend()
plt.show()

In [ ]:
%load_ext watermark
%watermark -v -m -iv